In [13]:
import requests
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import itertools
import warnings
import datetime
import pickle
# Import the statsmodels library for using SARIMAX model
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose

warnings.filterwarnings("ignore") # specify to ignore warning messages

In [2]:
# get the current date in the format yyyddmm
now_date = datetime.datetime.now()
now_date_str = str(now_date)
now_date_split = now_date_str.split('-')
now_day = now_date_split[2].split(' ')[0]
now_date_only = '{}{}{}'.format(now_date_split[0], now_date_split[1], now_day)
now_date_only

'20210729'

In [3]:
#download data for embu, nyeri aand nanyuki as test cities

def get_embu_data_from_2021():
    day_data_url = 'https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?&request=execute&identifier=SinglePoint&parameters=PRECTOT,T2M,WS2M&startDate=20200101&endDate=20210725&userCommunity=AG&tempAverage=DAILY&outputList=CSV&lat=-0.5423&lon=37.4543'
    response = requests.get(day_data_url)
    day_data = response.content.decode('utf-8')
    day_data = json.loads(day_data)
    features = day_data['features']
    features = features[0]
    properties = features['properties']
    parameters = properties['parameter']
    precipitation = parameters['PRECTOT'] #
    temp = parameters['T2M']
    wind = parameters['WS2M']
    
    return (precipitation, temp, wind)

def get_nyeri_data_from_2021():
    day_data_url = 'https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?&request=execute&identifier=SinglePoint&parameters=PRECTOT,T2M,WS2M&startDate=20200101&endDate=20210725&userCommunity=AG&tempAverage=DAILY&outputList=CSV&lat=-0.4371&lon=36.9580'
    response = requests.get(day_data_url)
    day_data = response.content.decode('utf-8')
    day_data = json.loads(day_data)
    features = day_data['features']
    features = features[0]
    properties = features['properties']
    parameters = properties['parameter']
    precipitation = parameters['PRECTOT'] #
    temp = parameters['T2M']
    wind = parameters['WS2M']
    
    return (precipitation, temp, wind)


def get_nanyuki_data_from_2021():
    day_data_url = 'https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?&request=execute&identifier=SinglePoint&parameters=PRECTOT,T2M,WS2M&startDate=20200101&endDate=20210725&userCommunity=AG&tempAverage=DAILY&outputList=CSV&lat=0.0074&lon=37.0722'
    response = requests.get(day_data_url)
    day_data = response.content.decode('utf-8')
    day_data = json.loads(day_data)
    features = day_data['features']
    features = features[0]
    properties = features['properties']
    parameters = properties['parameter']
    precipitation = parameters['PRECTOT'] #
    temp = parameters['T2M']
    wind = parameters['WS2M']
    
    return (precipitation, temp, wind)



In [4]:
#get the rainfall, temperature and wind for each of the test cities
print('Downloading data for Nyeri, Embu and Nanyuki as test citues')
embu_precipitation, embu_temp, embu_wind = get_embu_data_from_2021()
nyeri_precipitation, nyeri_temp, nyeri_wind = get_nyeri_data_from_2021()
nanyuki_precipitation, nanyuki_temp, nanyuki_wind = get_nanyuki_data_from_2021()

In [5]:
#convert the dictionaries to lists to form dataframes later

#for Embu
embu_dates = []
embu_temp_values = []
embu_precipitation_values = []
embu_wind_values = []

for key, value in embu_precipitation.items():
    embu_dates.append(key)
    embu_precipitation_values.append(value)
    
for key, value in embu_temp.items():
    embu_temp_values.append(value)
    
for key, value in embu_wind.items():
    embu_wind_values.append(value)
    
#for Nyeri
nyeri_dates = []
nyeri_temp_values = []
nyeri_precipitation_values = []
nyeri_wind_values = []

for key, value in nyeri_precipitation.items():
    nyeri_dates.append(key)
    nyeri_precipitation_values.append(value)
    
for key, value in nyeri_temp.items():
    nyeri_temp_values.append(value)
    
for key, value in nyeri_wind.items():
    nyeri_wind_values.append(value)
    
    
#for Nanyuki
nanyuki_dates = []
nanyuki_temp_values = []
nanyuki_precipitation_values = []
nanyuki_wind_values = []

for key, value in nanyuki_precipitation.items():
    nanyuki_dates.append(key)
    nanyuki_precipitation_values.append(value)
    
for key, value in nanyuki_temp.items():
    nanyuki_temp_values.append(value)
    
for key, value in nanyuki_wind.items():
    nanyuki_wind_values.append(value)
    

In [6]:
# form dataframes, for use later in the project for processing the data

embu_df = pd.DataFrame()
embu_df['datetime'] = embu_dates
embu_df['precipitation'] = embu_precipitation_values
embu_df['temp'] = embu_temp_values
embu_df['wind'] = embu_wind_values

nyeri_df = pd.DataFrame()
nyeri_df['datetime'] = nyeri_dates
nyeri_df['precipitation'] = nyeri_precipitation_values
nyeri_df['temp'] = nyeri_temp_values
nyeri_df['wind'] = nyeri_wind_values

nanyuki_df = pd.DataFrame()
nanyuki_df['datetime'] = nanyuki_dates
nanyuki_df['precipitation'] = nanyuki_precipitation_values
nanyuki_df['temp'] = nanyuki_temp_values
nanyuki_df['wind'] = nanyuki_wind_values

In [7]:
#reset indexes to datetime
embu_df.set_index('datetime', inplace=True)
nyeri_df.set_index('datetime', inplace=True)
nanyuki_df.set_index('datetime', inplace=True)

In [8]:
#Nyeri
# Shift the current temperature to the next day. 
embu_predicted_df = embu_df.temp.to_frame().shift(1).rename(columns = {"temp": "temp_pred" })
embu_actual_df = embu_df.temp.to_frame().rename(columns = {"temp": "temp_actual" })

# Concatenate the actual and predicted temperature
embu_one_step_df = pd.concat([embu_actual_df, embu_predicted_df],axis=1)

# Select from the second row, because there is no prediction for today due to shifting.
embu_one_step_df = embu_one_step_df[1:]


#Nyeri
# Shift the current temperature to the next day. 
nyeri_predicted_df = nyeri_df.temp.to_frame().shift(1).rename(columns = {"temp": "temp_pred" })
nyeri_actual_df = nyeri_df.temp.to_frame().rename(columns = {"temp": "temp_actual" })

# Concatenate the actual and predicted temperature
nyeri_one_step_df = pd.concat([nyeri_actual_df, nyeri_predicted_df],axis=1)

# Select from the second row, because there is no prediction for today due to shifting.
nyeri_one_step_df = nyeri_one_step_df[1:]


#Nyeri
# Shift the current temperature to the next day. 
nanyuki_predicted_df = nanyuki_df.temp.to_frame().shift(1).rename(columns = {"temp": "temp_pred" })
nanyuki_actual_df = nanyuki_df.temp.to_frame().rename(columns = {"temp": "temp_actual" })

# Concatenate the actual and predicted temperature
nanyuki_one_step_df = pd.concat([nanyuki_actual_df, nanyuki_predicted_df],axis=1)

# Select from the second row, because there is no prediction for today due to shifting.
nanyuki_one_step_df = nanyuki_one_step_df[1:]



In [9]:
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [10]:
'''
#for embu
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(embu_one_step_df.temp_actual,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('SARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue
            
            
# for nyeri          
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(nyeri_one_step_df.temp_actual,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('SARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue


#for nanyuki            
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(nanyuki_one_step_df.temp_actual,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('SARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue
            
'''            

"\n#for embu\nfor param in pdq:\n    for param_seasonal in seasonal_pdq:\n        try:\n            mod = sm.tsa.statespace.SARIMAX(embu_one_step_df.temp_actual,\n                                            order=param,\n                                            seasonal_order=param_seasonal,\n                                            enforce_stationarity=False,\n                                            enforce_invertibility=False)\n\n            results = mod.fit()\n\n            print('SARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))\n        except:\n            continue\n            \n            \n# for nyeri          \nfor param in pdq:\n    for param_seasonal in seasonal_pdq:\n        try:\n            mod = sm.tsa.statespace.SARIMAX(nyeri_one_step_df.temp_actual,\n                                            order=param,\n                                            seasonal_order=param_seasonal,\n                                            enforce_stati

In [11]:
#embu
# Fit the SARIMAX model using optimal parameters
print('Trainign the Embu weather Model')
embu_mod = sm.tsa.statespace.SARIMAX(embu_one_step_df.temp_actual,
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

embu_results = embu_mod.fit()


#nyeri
# Fit the SARIMAX model using optimal parameters
print('Training the Nyeri waether Model')
nyeri_mod = sm.tsa.statespace.SARIMAX(nyeri_one_step_df.temp_actual,
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

nyeri_results = nyeri_mod.fit()


#nanyuki
# Fit the SARIMAX model using optimal parameters
print('Training the Nayuki weather Model')
nanyuki_mod = sm.tsa.statespace.SARIMAX(nanyuki_one_step_df.temp_actual,
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

nanyuki_results = nanyuki_mod.fit()

print('Hoooooorey! Finished Training all the models, Now saving them')

Trainign the Embu weather Model
Training the Nyeri waether Model
Training the Nayuki weather Model
Hoooooorey! Finished Training all the models, Now saving them


In [109]:
embu_model_name = 'embu_weather_model'
nyeri_model_name = 'nyeri_weather_model'
nanyuki_model_name = 'nayuki_weather_model'

embu_results.save(embu_model_name, remove_data=False)
nyeri_results.save(nyeri_model_name, remove_data=False)
nanyuki_results.save(nanyuki_model_name, remove_data=False)

print('You just  saved the models and reached the end of the training, its time to do other things!')

You just  saved the models and reached the end of the training, its time to do other things!


In [17]:
nyeri_model_name = 'nyeri_weather_model' 
pickle.dump(nyeri_results, open(nyeri_model_name, 'wb'))

In [18]:
nayuki_model_name = 'nanyuki_weather_model' 
pickle.dump(embu_results, open(nayuki_model_name, 'wb'))

In [19]:
embu_model_name = 'embu_weather_model' 
pickle.dump(embu_results, open(embu_model_name, 'wb'))

In [20]:
loaded_model = pickle.load(open(embu_model_name, 'rb'))
pred = loaded_model.get_prediction(start=pd.to_datetime('20210803'), dynamic=False)
pred_ci = pred.conf_int()
pred_ci

,lower temp_actual,upper temp_actual
2021-08-03,14.817879,18.326054


In [21]:
# load the embu model and try predicting
embu_weather_model = sm.load('embu_weather_model')

pred = embu_weather_model.get_prediction(start=pd.to_datetime('20210803'), dynamic=False)
pred_ci = pred.conf_int()
pred_ci

,lower temp_actual,upper temp_actual
2021-08-03,14.817879,18.326054


In [111]:
# load the embu model and try predicting
nyeri_weather_model = sm.load('nyeri_weather_model')

pred = nyeri_weather_model.get_prediction(start=pd.to_datetime('20210803'), dynamic=False)
pred_ci = pred.conf_int()
pred_ci

,lower temp_actual,upper temp_actual
2021-08-03,13.027526,15.998946


In [112]:
# load the embu model and try predicting
nayuki_weather_model = sm.load('nayuki_weather_model')

pred = nayuki_weather_model.get_prediction(start=pd.to_datetime('20210803'), dynamic=False)
pred_ci = pred.conf_int()
low_temp = pred_ci['lower temp_actual'].tolist()[0]
high_temp = pred_ci['upper temp_actual'].tolist()[0] 
mean_temp = (low_temp+high_temp)/2
print('The lowest temp {}, the highest temp {}, and the mean temp is {}'.format(low_temp, high_temp, mean_temp))

The lowest temp 16.109482893320624, the highest temp 19.376556090960847, and the mean temp is 17.743019492140736


In [113]:
city = 'nyerI'
print(city.capitalize())

Nyeri
